In [1]:
import pandas as pd
import regex as re
import emoji
import string
# Malay NLP #

import malaya
import logging

#Spelling Correction
import tensorflow as tf
import symspellpy

# Normalization #

from pprint import pprint
#translation
from googletrans import Translator

translator = Translator()

#Custom Dictionary
from python.customDict import getMyDict


C:\Users\General IRIS\PycharmProjects\FinalYearProject\venv\lib\site-packages\malaya_boilerplate\frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(
C:\Users\General IRIS\PycharmProjects\FinalYearProject\venv\lib\site-packages\malaya_boilerplate\frozen_graph.py:38: UserWarning: check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
  warnings.warn(
C:\Users\General IRIS\PycharmProjects\FinalYearProject\venv\lib\site-packages\malaya\tokenizer.py:208: FutureWarning: Possible nested set at position 3372
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
C:\Users\General IRIS\PycharmProjects\FinalYearProject\venv\lib\site-packages\malaya\tokenizer.py:208: FutureWarning: Possible nested set 

# DATA CLEANING MALAY


In [2]:
df = pd.read_csv('CurrentData.csv', encoding='utf-8')

### Functions Only

In [3]:
def dataChecking(df,num):

    if num == 1:
        print(df.shape[0])
    elif num == 2:
        print(df.isnull().sum())
    print("\n")

def convertToStr(somestring):

    properString = ''

    if type(somestring) != str:
        properString = str(somestring)
    else:
        properString = somestring

    return properString

def countEmoji(aString):

    #\X to check individual emoji where family emoji is actually not 1 emoji but multiple ~5
    # Hence calculate it as 1 not lets say 5
    data = re.findall(r'\X', aString.strip())

    return sum(emoji.is_emoji(word) for word in data)


def countPunct(aString):

    data = re.findall(r'\X', aString.strip())
    return sum(word in string.punctuation for word in data)

def countCaps(aString):
    return len(re.findall(r'[A-Z]', aString.strip()))


def removeTooLongAndSeparateNumAndWord(comment,tokenizer):
    extraWords = getMyDict()

    comment = comment.lower()

    tokenized = tokenizer.tokenize(comment)
    reducedWords = []
    #Remove extra lengthy words + seperate numbers and letters
    for i, val in enumerate(tokenized):
        if len(val) < 20:
            result = re.findall('\d+|\D+', val)
            reducedWords = reducedWords + result

    #Replace and fix words (Manually)
    for i in range( len(reducedWords) ):

        text = reducedWords[i]
        if text.isnumeric():
            continue
        try:
            if (reducedWords[i] == 't'):
                reducedWords[i] = ""

            else:
                reducedWords[i] = extraWords[text]
        except Exception:
            pass

    reducedSentences = ' '.join(reducedWords).strip()

    SimpleCommentManglish = re.sub( r'[^A-Za-z0-9]+',' ', reducedSentences)
    return SimpleCommentManglish


#Preprocessing + Normalization
def spellerCorrector(comment,model):
    #Tries fix simple words

    if comment:
        commentCorrected =  ' '.join(list(map(lambda each:each.replace('<repeated>','').replace('</repeated>','').replace('<elongated>','').replace('</elongated>','').replace('<number>','').replace('</number>',''), model.process(comment))))
        return ' '.join( commentCorrected.split() )
    return ''


# Remove elongated words
def malayNormalizer(comment,normalizer):
    try:

        if comment or comment != 'None' or comment != "":

            newComment = normalizer.normalize(comment,normalize_entity = False)
            return newComment['normalize']

        return ''
    except Exception:
        #Debug
        print("Exception Here\n\n" + comment)
        return comment



def deepSpellerCorrector(comment,model):

    #Tries to correct short forms and slangs words
    try:
        if comment or comment != 'None' or comment != "":

            return model.correct_text(comment)

        return ''
    except Exception:
        print("Exception Here\n\n" + comment)
        return comment



def malayToEnglishTranslator(comment, model):
    #Finally change BM -> ENG
    if comment:
        return ' '.join(model.greedy_decoder([comment]))

    return ''



def usingGoogleTranslate(comment):
    if comment:
        r = translator.translate(comment, src='ms', dest = 'en')
        return r.text

    return ''



In [4]:
df.head(50)

,Username,Userid,Rating,Helpfulness,Comment,TimeCreated,Target
0,i*****_,218195265,3,NaN,SSD boleh pakai cuma...bila dah transfer tetib...,2/25/2022 22:20,1
1,simonchan1905,705330191,1,1.0,Never trust or purchase usb or sd card if the ...,2022-07-29 08:29:09,1
2,ladydelilah,90528909,5,NaN,"Alhamdulillah dh smpi, terus guna.. saje amek ...",2022-01-07 11:28:36,0
3,anna_aizad,34734312,1,7.0,All my file corrupted after saving a few file ...,2022-07-13 14:28:30,1
4,b*****d,758613696,5,2.0,"Box came bubble-wrapped, so everything is in g...",2022-07-12 13:33:12,0
5,beskotkebeng,184848780,5,2.0,Copy speed based on your internal storage. Man...,2020-10-04 09:57:55,1
6,mikrobistro,554068149,2,3.0,Barang sampai dlm keadaan baik tp kenapa hard ...,5/19/2022 12:27,1
7,suraya0806,30636902,5,NaN,Received item in good condition. Secure packag...,2021-08-18 19:00:41,0
8,aimestudio,39266309,5,NaN,Good Product Quality Seagate ✅\nLaju. if nak s...,2021-06-28 03:17:32,1
9,j*****n,219751578,5,NaN,Item received in good condition but yet to be ...,2021-06-22 20:41:30,0


# PHASE 1
## Data Cleaning Malay Part 1
1. Removing Nulls [Comment + Helpfulness]
2. Convert All Values to String
3. Filter only letters A-z 0 -9 only ;)

## Feature Engineering Part 1
1. Calc Length Words in Original Comment [Manglish]
2. Calc Total Emoji ;)
3. Calc Total Punctuation !!!!@!#!$!$....??
4. Calc Total Capital Letters ASDAJNDOADNKkmajksdmad


In [5]:
%%time
def cleaning(df):

    #View overall original data
    #dataChecking(df,1)

    #Remove NaN and normalize {} to 0 for helpfulness
    df['Comment'] = df['Comment'].replace('nan','NaN')
    df['Helpfulness'] = df['Helpfulness'].fillna(0)


    #View total empties before and after
    #dataChecking(df,2)
    df = df.dropna()
    #dataChecking(df,2)


    #Possibilty that some data may recognize as integers hence convrt -> string
    df['CommentToStr'] = df['Comment'].apply(lambda x : convertToStr(x))


    #removing def punc {!,$,#,%,^}
    #[^\w\s]
    #remove everything for text process only
    #[^A-Za-z0-9]+
    tokenizer = malaya.tokenizer.Tokenizer()
    df['SimpleCommentManglish'] = df['CommentToStr'].apply(lambda x: removeTooLongAndSeparateNumAndWord(x,tokenizer) )


    #Calc Length of words
    df['LengthManglish'] = df['SimpleCommentManglish'].apply(lambda x : len(x.split(' ')))


    #Calc Emoji
    df['TotalEmoji'] = df['CommentToStr'].apply(lambda x : countEmoji(x))


    #Calc Punct
    df['TotalPunct'] = df['CommentToStr'].apply(lambda x : countPunct(x))


    #Calc Caps
    df['TotalCaps'] = df['CommentToStr'].apply(lambda x : countCaps(x))

    return df

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
df  = cleaning(df)
#df.head(50)

# PHASE 2
## Data Cleaning Malay Part 2 [Final]
1. Remove Elogated [bagussssss = bagus] & Repeated [bagus bagus bagus bagus = bagus]
2. Normalize The Words [ko = engkau]
3. Speller Corrector Malay [sbg = sebagai] If can't be corrected by Normalizer, Quite Long ;(
4. Translation Malay -> English

In [7]:
%%time
def preprocessing(df):
    # Remove GPU & with to return to normal V2 Tensors

    preprocessModel = malaya.preprocessing.preprocessing(normalize=['number'], annotate=['elongated', 'repeated'])
    df['CommentClean1SimpleSpellCorrector'] = df['SimpleCommentManglish'].apply(lambda x : spellerCorrector(x,preprocessModel))
    print("Simple Adjuster Done")


    normalizer = malaya.normalize.normalizer()
    df['CommentClean2Normalizing'] = df['CommentClean1SimpleSpellCorrector'].apply(lambda x : malayNormalizer(x,normalizer))
    print("Normalizer Done")

    # Use  En + M Normalize
    # Old
    """normalizer = malaya.normalize.normalizer(malaya.spell.probability())
    df['CommentClean2'] = df['CommentClean1'].apply(lambda x : malayNormalizer(x,normalizer))
    print("Normalizer Done")"""

    """model = malaya.transformer.load

    (model = 'electra')
    transformer_corrector = malaya.spelling_correction.transformer.encoder(model, sentence_piece = True)
    df['CommentClean3'] = df['CommentClean2'].apply(lambda x : deepSpellerCorrector(x,transformer_corrector))"""

    """
    # New
    symspell_corrector = malaya.spelling_correction.symspell.load()
    df['CommentClean3DeepSpellCorrector'] = df['CommentClean2Normalizing'].apply(lambda x : deepSpellerCorrector(x,symspell_corrector))
    print("Deep Spell Done")
    """

    # Translators

    modelMalayEnglishTranslator = malaya.translation.ms_en.transformer(model= 'noisy-base')
    df['CommentClean4EnglishComment'] = df['CommentClean2Normalizing'].apply(lambda x : malayToEnglishTranslator(x,modelMalayEnglishTranslator).lower())

    #tf.compat.v2.keras.backend.clear_session()
    """
    df['CommentClean4EnglishComment'] = df['CommentClean2Normalizing'].apply(lambda x : usingGoogleTranslate(x))
    print("Language Translated")
    """

    return df

CPU times: total: 0 ns
Wall time: 0 ns


In [8]:
%%time
df = preprocessing(df)
#df.head(50)

Simple Adjuster Done
Normalizer Done
CPU times: total: 18min 48s
Wall time: 13min 56s


# Export Result

In [9]:
df.to_csv("TestOnly.csv", encoding='utf-8', index=False, mode='a')